In [1]:
import gensim
gensim.__version__

C:\Users\Armik\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


'3.6.0'

In [2]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
from preprocess_func import morpher, sw, exclude, preprocess_txt, embed_txt

In [4]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
#Small preprocess of the answers

question = None
written = False

with open("../data/prepared_answers.txt", "w", encoding='utf-8') as fout:  
    with open("../data/Otvety.txt", "r", encoding='utf-8') as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 
Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте

In [6]:
# Preprocess for models fitting

sentences = []

c = 0

with open("../data/Otvety.txt", "r", encoding='utf-8') as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 50000:
            break

0it [00:00, ?it/s]

In [7]:
sentences = [i for i in sentences if len(i) > 2]


In [8]:
len(sentences)

32310

In [9]:
# Классифицируем текст с помощью FastText
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5) # gensim == 4.3.1 vector_size
modelFT.save("../data/bot_trained/ft_model")

In [10]:
# Создаем Индексы для вопросов-ответов

#modelFT = FastText.load("../data/bot_trained/ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [11]:
index_map = {}
counter = 0

with open("../data/prepared_answers.txt", "r", encoding='utf-8') as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

        if counter > 50000:
            break

ft_index.build(10)
ft_index.save('../data/bot_trained/speaker.ann')

0it [00:00, ?it/s]

True

In [12]:
#ft_index = annoy.AnnoyIndex(100, 'angular')
#ft_index.load('speaker.ann') 

In [13]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[51, 315]

In [14]:
with open("../data/bot_trained/index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [15]:
with open("../data/bot_trained/index_speaker.pkl", "rb") as f:
    index_loaded = pickle.load(f)

In [16]:
# checking loaded from pickle index
index_loaded[5]

'Защита мыслью. <br>Каждый человек должен в отношении вампиров взять для себя за правило: "Нам не страшен серый волк", то есть знать, что конкретный "серый волк" - вампир существует, но воспринимать его следует не как хищника, а как добрую домашнюю собачку. Такая "собачка" быстро почувствует, что вы не "того поля ягода" и быстро отстанет сама. <br>"Витание в облаках". <br>Тоже хороший способ. Мысленно представить себя находящимся в окружении приятных нашим ощущениям белых облаков или сине-голубого неба. Это просто и эффективно. <br>"Подкормка негативом". <br>Вампир хочет кушать? Так дайте ему. Соберите в себе все плохие мысли, эмоции, все то отрицательное, что каким-то образом закралось в организм. И всю эту "тяжесть" отдайте вампиру - пусть подавится. <br>Зеркало <br>При встрече или при общении с неприятным человеком мысленно представить, что между вами зеркальная стена, обращенная отражающей поверхностью к этому человеку. <br>"Дикобраз" <br>Этот способ выглядит следующим образом: чел